In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')
 
data = wine[['alcohol', 'sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size = 0.2,random_state = 42)

In [ ]:
sub_input,val_input,sub_target,val_target = train_test_split(train_input, train_target,test_size = 0.2, random_state = 42)

In [ ]:
print(sub_input.shape , val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.00714111, 0.0068841 , 0.00714755, 0.00705719, 0.00673175]), 'score_time': array([0.00061607, 0.00054145, 0.00052309, 0.00054979, 0.00051045]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
from sklearn.model_selection import cross_val_score
scores2 = cross_val_score(dt,train_input,train_target)
print(scores2)

[0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
split = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = split)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs = -1)

In [ ]:
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
dt2 = DecisionTreeClassifier(max_depth = 14, min_impurity_decrease=0.0004,min_samples_split= 12, random_state= 42)
dt2.fit(train_input,train_target)
print(dt2.score(train_input,train_target))

0.892053107562055


In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0,10)
rgen.rvs(10)

array([6, 8, 9, 0, 9, 3, 9, 0, 1, 6])

In [ ]:
np.unique(rgen.rvs(1000),return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([116, 106,  92,  95, 103,  83, 107, 111,  84, 103]))

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs =RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter = 'random'),params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [ ]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
